In [1]:
import numpy as np
from typing import Optional, Tuple, Union
import matplotlib.pyplot as plt
from scipy import signal
from glob import glob
from PIL import Image

In [2]:
plt.rcParams["font.family"] = "serif"
plt.rcParams["font.size"] = 16

In [3]:
class Life:
    """
    Class simulating Conway's Game of Life
    """
    def __init__(
        self,
        primordial_soup: Optional[np.ndarray] = None,
        board_size: Union[int, Tuple[int, int]] = 100,
        random_seed: int = 12,
        cell_fraction: float = 0.75,
        generation: int = 0,
    ):
        """
        :param primordial_soup: A numpy array representing the intial state of the game, if not provided a randomized board will be generated
        :param board_size: The initial size of the board (rows, columns), if provided as an int the initial board will be square (NxN)
        :param random_seed: Random seed provided to numpy
        :param cell_fraction: Fraction of cells that begin alive
        """
        np.random.seed(random_seed)
        self.generation = generation
        
        # If an initialization array is provided, use its shape, otherwise set the board shape based on the size parameter
        if primordial_soup is not None:
            self.board_shape = primordial_soup.shape
        elif isinstance(board_size, tuple):
            self.board_shape = board_size
        else:
            self.board_shape = (board_size, board_size)

        self.cell_fraction = cell_fraction
        self.board_state = primordial_soup if primordial_soup is not None else self.generate_randomized_board()
        
    def generate_randomized_board(self):
        """
        Generate a board of shape self.board_shape with a number of living cells based on self.cell_fraction
        """
        # Randomize the board with random numbers between 0 and 1
        # Set a living cell threshold based on the desired fraction of cells
        randomized_board = np.random.uniform(0, 1, self.board_shape)
        living_threshold = 1 - self.cell_fraction
        
        # If a board entry is above threshold, set it to living
        # If a board entry is below threshold, set it to dead
        randomized_board[randomized_board >= living_threshold] = 1
        randomized_board[randomized_board < living_threshold] = 0
        return randomized_board
    
    def step_board_state(self):
        """
        Evolve the board by one step by comparing each cell to its number of living neighbors and updating its state.
        
        The rules:
          * Any living cell with 2 or 3 neighbors lives on
          * Any living cell with less than 2 neighbors dies
          * Any living cell with more than 3 neighbors dies
          * Any dead cell with exactly 3 neighbors is resurrected
        """
        # Look at each cell's neighbor to generate a matrix where each value represents the number of
        # adjacent living cells
        neighbor_array = self.generate_neighbor_array()

        # Isolate the living cells and apply the rule set to them using a piecewise function
        living_cells = np.multiply(self.board_state, neighbor_array)
        surviving_cells = np.piecewise(
            living_cells,
            [living_cells == 2, living_cells == 3, living_cells < 2, living_cells > 3],
            [1, 1, 0, 0],
        )
        
        # Isolate the dead cells with many neighbors and resurrect them
        surviving_cells[(self.board_state == 0) & (neighbor_array == 3)] = 1
        self.board_state = surviving_cells
        self.generation += 1
        
    def display_board_state(self, axis):
        """
        Draw board state on to a provided matplotlib axis
        """
        axis.imshow(self.board_state, cmap="bone_r")
        axis.get_xaxis().set_visible(False)
        axis.get_yaxis().set_visible(False)
        axis.set_title(f"Generation: {self.generation}")
        
    def generate_neighbor_array(self):
        """
        Convolve the board state with an array of:
        1 1 1 
        1 0 1
        1 1 1
        to count the neighbors
        """
        sliding_window = np.ones((3,3))
        sliding_window[1,1] = 0
        neighbors = signal.convolve(self.board_state, sliding_window, mode="same")
        return neighbors
    
    def contaminate(self, new_cells: int = 1, in_place=False):
        """
        Introduce a number of new cells to randomly contaminate the population and return a new experiment
        """
        # Produce a list of all possible cell locations, randomize, and select a number of them to be contaminated
        possible_locations = np.array(list(np.ndindex(self.board_shape)))
        np.random.shuffle(possible_locations)
        cells_to_fill = possible_locations[:new_cells]
        
        # Copy the board state, fill in the randomized cells, and return a new instance of Life initialized
        # with the updated state
        if in_place:
            self.board_state[tuple(cells_to_fill.T)] = 1
            return
        else:
            contaminated_state = np.copy(self.board_state)
            contaminated_state[tuple(cells_to_fill.T)] = 1
            return Life(primordial_soup=contaminated_state, generation=self.generation)

## Create a board and evolve it for 60 generations

In [4]:
t=Life(board_size=(20, 30), cell_fraction=0.15)
while t.generation < 60:
    board_fig = plt.figure(figsize=(5,5), dpi=150)
    t.display_board_state(axis=board_fig.gca())
    board_fig.savefig(f"life_simulation/standard/img_{t.generation:03d}.png")
    plt.close()
    t.step_board_state()
    
img, *imgs = [Image.open(f) for f in sorted(glob("life_simulation/standard/img_*.png"))]
img.save(fp="life_simulation/standard/evolution.gif", format='GIF', append_images=imgs,
         save_all=True, duration=200, loop=0, dpi=150)

## Create a version that contaminates the sample halfway through the evolution

In [5]:
t_clean=Life(board_size=(20, 30), cell_fraction=0.15)
t_contaminated=Life(board_size=(20, 30), cell_fraction=0.15)

while t_clean.generation < 120:
    # Contaminate the second sample after 30 generations
    if t_clean.generation == 30:
        t_contaminated.contaminate(new_cells=15, in_place=True)
    
    # plot the two samples side by side
    board_fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,5), dpi=150)
    t_clean.display_board_state(axis=ax1)
    t_contaminated.display_board_state(axis=ax2)
    ax1.annotate("Clean", xy=(0.5, -0.05), va="top", ha="center", xycoords="axes fraction")
    ax2.annotate("Contaminated", xy=(0.5, -0.05), va="top", ha="center", xycoords="axes fraction")
    board_fig.savefig(f"life_simulation/comparison/img_{t_clean.generation:03d}.png")
    plt.close()

    t_clean.step_board_state()
    t_contaminated.step_board_state()
    
img, *imgs = [Image.open(f) for f in sorted(glob("life_simulation/comparison/img_*.png"))]
img.save(fp="life_simulation/comparison/evolution.gif", format='GIF', append_images=imgs,
         save_all=True, duration=200, loop=0)

## Add a glider!

In [6]:
glider_board = np.zeros((30,30))
glider = np.array([
    [0, 0, 1],
    [1, 0, 1],
    [0, 1, 1],
])
glider_board[0:3, 0:3] = glider

In [7]:
glider_game = Life(primordial_soup=glider_board)

In [8]:
while glider_game.generation < 60:
    board_fig = plt.figure(figsize=(5,5), dpi=150)
    glider_game.display_board_state(axis=board_fig.gca())
    board_fig.savefig(f"life_simulation/glider/img_{glider_game.generation:03d}.png")
    plt.close()
    glider_game.step_board_state()
    
img, *imgs = [Image.open(f) for f in sorted(glob("life_simulation/glider/img_*.png"))]
img.save(fp="life_simulation/glider/evolution.gif", format='GIF', append_images=imgs,
         save_all=True, duration=200, loop=0)